# Visual Data Analysis of Fraudulent Transactions

Your CFO has also requested detailed trends data on specific card holders. Use the starter notebook to query your database and generate visualizations that supply the requested information as follows, then add your visualizations and observations to your markdown report.

In [1]:
# Initial imports
import pandas as pd
import calendar
import hvplot.pandas
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load .env enviroment variables
load_dotenv()


True

In [2]:
# Create a connection to the database
database_url = os.getenv("database_url")
engine = create_engine(database_url)


## Data Analysis Question 1

The two most important customers of the firm may have been hacked. Verify if there are any fraudulent transactions in their history. For privacy reasons, you only know that their cardholder IDs are 2 and 18.

* Using hvPlot, create a line plot representing the time series of transactions over the course of the year for each cardholder separately. 

* Next, to better compare their patterns, create a single line plot that containins both card holders' trend data.  

* What difference do you observe between the consumption patterns? Does the difference suggest a fraudulent transaction? Explain your rationale in the markdown report.

In [3]:
# loading data for card holder 2 and 18 from the database
# Write the query
query = """

SELECT
    t.date AS date,
    ch.id AS card_holder_id, 
    cc.card AS credit_card_number,
    t.amount AS amount,
    t.id AS transaction_id, 
    m.id AS merchant_id,
    m.name AS merchant_name,
    mc.name AS merchant_category_name
FROM card_holder ch
JOIN credit_card cc ON ch.id = cc.cardholder_id
JOIN transaction t ON cc.card = t.card
LEFT JOIN merchant m ON t.id_merchant = m.id
LEFT JOIN merchant_category mc ON m.id_merchant_category = mc.id
WHERE ch.id IN (2, 18);

        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
card_holder_2_18 = pd.read_sql(query, engine)


# Clean Data and Prepare 
card_holder_2_18.isnull().count()
card_holder_2_18.dropna()
card_holder_2_18['date'] = pd.to_datetime(card_holder_2_18['date'])
card_holder_2_18['amount'] = card_holder_2_18['amount'].str.replace('[$,]', '', regex=True)
card_holder_2_18['amount'] = pd.to_numeric(card_holder_2_18['amount'], errors='coerce')

display(card_holder_2_18.head())
display(card_holder_2_18.tail())

,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category_name
0,2018-01-01 23:15:10,18,4498002758300,2.95,567,64,"Cline, Myers and Strong",restaurant
1,2018-01-05 07:19:27,18,344119623920892,1.36,2077,30,Atkinson Ltd,bar
2,2018-01-06 02:16:41,2,4866761290278198714,1.33,2439,127,"Williams, Wright and Wagner",coffee shop
3,2018-01-06 05:13:20,2,4866761290278198714,10.82,1867,70,White-Hall,bar
4,2018-01-07 01:10:54,18,344119623920892,175.00,3457,12,"Bell, Gonzalez and Lowe",pub


,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category_name
227,2018-12-27 18:46:57,18,344119623920892,1.70,1994,55,"Johnson, Rivas and Anderson",restaurant
228,2018-12-28 08:45:26,18,4498002758300,3.46,114,82,Day-Murray,food truck
229,2018-12-28 09:00:45,18,344119623920892,12.88,1228,60,Smith-Richards,pub
230,2018-12-28 15:30:55,2,675911140852,11.03,962,2,Riggs-Adams,restaurant
231,2018-12-29 08:11:55,18,4498002758300,12.25,1218,6,Wilson and Sons,restaurant


In [4]:
# Plot for cardholder 2
card_holder_2 = card_holder_2_18[card_holder_2_18['card_holder_id']==2]
line_plot_2 = card_holder_2.hvplot.line(x='date', y='amount', title='Transaction History', height=400, width=700)
line_plot_2

:Curve   [date]   (amount)

In [5]:
# Count amount of suspected fraudulent activities <$2 and >$500
Fraud_detection_lower_ch2 = card_holder_2[card_holder_2['amount'] < 2]
Fraud_detection_higher_ch2 = card_holder_2[card_holder_2['amount'] > 500]
lower = Fraud_detection_lower_ch2['amount'].count()
higher = Fraud_detection_higher_ch2['amount'].count()

print(f"fraudulent activities detected: {lower} transactions lower than $2")
print(f"fraudulent activities detected: {higher} transactions Higher than $500")

fraudulent activities detected: 11 transactions lower than $2
fraudulent activities detected: 0 transactions Higher than $500


In [6]:
# Plot for cardholder 18
card_holder_18 = card_holder_2_18[card_holder_2_18['card_holder_id']==18]
line_plot_18 = card_holder_18.hvplot.line(x='date', y='amount', title='Transaction History', height=400, width=700)
line_plot_18

:Curve   [date]   (amount)

In [7]:
# Count amount of suspected fraudulent activities <$2 and >$500
Fraud_detection_lower_ch18 = card_holder_18[card_holder_18['amount'] < 2]
Fraud_detection_higher_ch18 = card_holder_18[card_holder_18['amount'] > 500]
lower = Fraud_detection_lower_ch18['amount'].count()
higher = Fraud_detection_higher_ch18['amount'].count()

print(f"fraudulent activities detected: {lower} transactions lower than $2")
print(f"fraudulent activities detected: {higher} transactions Higher than $500")

fraudulent activities detected: 19 transactions lower than $2
fraudulent activities detected: 7 transactions Higher than $500


In [8]:
# Combined plot for card holders 2 and 18
combined_plot = line_plot_2 * line_plot_18
combined_plot

:Overlay
   .Curve.I  :Curve   [date]   (amount)
   .Curve.II :Curve   [date]   (amount)

## Data Analysis Question 2

The CEO of the biggest customer of the firm suspects that someone has used her corporate credit card without authorization in the first quarter of 2018 to pay quite expensive restaurant bills. Again, for privacy reasons, you know only that the cardholder ID in question is 25.

* Using hvPlot, create a box plot, representing the expenditure data from January 2018 to June 2018 for cardholder ID 25.

* Are there any outliers for cardholder ID 25? How many outliers are there per month?

* Do you notice any anomalies? Describe your observations and conclusions in your markdown report.

In [9]:
# loading data of daily transactions from jan to jun 2018 for card holder 25
# Write the query
query = """

SELECT 
    t.date, 
    ch.id AS card_holder_id, 
    cc.card AS credit_card_number,
    t.amount,
    t.id AS transaction_id,
    m.id AS merchant_id,
    m.name AS merchant_name,
    mc.name AS merchant_category
FROM transaction t
JOIN credit_card cc ON t.card = cc.card
JOIN card_holder ch ON cc.cardholder_id = ch.id
JOIN merchant m ON t.id_merchant = m.id
LEFT JOIN merchant_category mc ON m.id_merchant_category = mc.id
WHERE ch.id = 25 AND t.date BETWEEN '2018-01-01' AND '2018-06-30'
ORDER BY t.date;

        """
# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
card_holder_25 = pd.read_sql(query, engine)

# Clean Data and Prepare 
card_holder_25.isnull().count()
card_holder_25.dropna()
card_holder_25['date'] = pd.to_datetime(card_holder_25['date'])
card_holder_25['amount'] = card_holder_25['amount'].str.replace('[$,]', '', regex=True)
card_holder_25['amount'] = pd.to_numeric(card_holder_25['amount'], errors='coerce')

display(card_holder_25.head())
display(card_holder_25.tail())

,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category
0,2018-01-02 02:06:21,25,4319653513507,1.46,2083,93,Rodriguez-Parker,food truck
1,2018-01-05 06:26:45,25,372414832802279,10.74,1552,86,"Walker, Campbell and Sullivan",food truck
2,2018-01-07 14:57:23,25,4319653513507,2.93,2108,137,Garcia PLC,food truck
3,2018-01-10 00:25:40,25,372414832802279,1.39,754,50,Johnson-Watts,restaurant
4,2018-01-14 05:02:22,25,372414832802279,17.84,3023,52,Jensen-Stanley,food truck


,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category
62,2018-06-19 07:16:54,25,372414832802279,15.98,3230,47,Martin Inc,restaurant
63,2018-06-22 06:16:50,25,4319653513507,1813.00,2582,40,"Cox, Montgomery and Morgan",bar
64,2018-06-23 22:36:00,25,4319653513507,16.61,3218,144,"Walker, Deleon and Wolf",restaurant
65,2018-06-25 09:17:05,25,372414832802279,11.53,1523,77,"Brown, Ballard and Glass",restaurant
66,2018-06-27 14:33:06,25,372414832802279,5.24,2264,26,Smith-Stephens,coffee shop


In [10]:
# loop to change the numeric month to month names
for month in range(len(card_holder_25)):
    month_num = card_holder_25.loc[month, 'date'].month
    card_holder_25.loc[month, 'month'] = calendar.month_name[month_num]
    
display(card_holder_25.head())
display(card_holder_25.tail())

,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category,month
0,2018-01-02 02:06:21,25,4319653513507,1.46,2083,93,Rodriguez-Parker,food truck,January
1,2018-01-05 06:26:45,25,372414832802279,10.74,1552,86,"Walker, Campbell and Sullivan",food truck,January
2,2018-01-07 14:57:23,25,4319653513507,2.93,2108,137,Garcia PLC,food truck,January
3,2018-01-10 00:25:40,25,372414832802279,1.39,754,50,Johnson-Watts,restaurant,January
4,2018-01-14 05:02:22,25,372414832802279,17.84,3023,52,Jensen-Stanley,food truck,January


,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category,month
62,2018-06-19 07:16:54,25,372414832802279,15.98,3230,47,Martin Inc,restaurant,June
63,2018-06-22 06:16:50,25,4319653513507,1813.00,2582,40,"Cox, Montgomery and Morgan",bar,June
64,2018-06-23 22:36:00,25,4319653513507,16.61,3218,144,"Walker, Deleon and Wolf",restaurant,June
65,2018-06-25 09:17:05,25,372414832802279,11.53,1523,77,"Brown, Ballard and Glass",restaurant,June
66,2018-06-27 14:33:06,25,372414832802279,5.24,2264,26,Smith-Stephens,coffee shop,June


In [11]:
# Creating the six box plots using hvPlot
box_plot_25 = card_holder_25.hvplot.box(y='amount', by='month', xlabel='Month', ylabel='Amount', title='Monthly Transaction Amounts')
box_plot_25

:BoxWhisker   [month]   (amount)